## Trying to do fragpipe analyst R in python 
due to intensive package downlaoding: it takes too long even on HPC

FragPipe-Analyst and FragPipeAnalystR share the same common modular design with the following modules: \
(1) I/O module that handles the input and output files. \
(2) Data manipulation module, which has functions for operating the fundamental data structure, including removing/merging samples and feature selections. \
(3) Data filtering module, which provides methods for filtering data based on missing values. \
(4) Normalization module that provides several normalization methods. \
(5) Imputation module that provides several imputation methods. \
(6) Quality control (QC) module, which provides functions to generate various visualizations, including principal component analysis (PCA) plots and heatmaps. \
(7) Differential expression (DE) analysis module, which provides functions for performing statistical procedures and result visualization, such as a volcano plot. \
(8) Enrichment analysis module, which provides statistical procedures specifically for inferring biological insights, such as overrepresentation tests.\


1. Import tsv from FP output

In [2]:
import pandas as pd
import numpy as np

In [8]:
combined_protein = pd.read_csv(r"data_WTvs3PA/combined_protein.tsv", header=0, sep="\t")
annotation = pd.read_csv(r"data_WTvs3PA/experiment_annotation.tsv", header=0, sep="\t")


In [ ]:
combined_protein=combined_protein.set_index('Entry Name')

#filter(regex=...) searches column names
#(?i) makes it case‑insensitive
#Returns only the matching columns
maxlfq_cols = combined_protein.filter(regex='(?i)MaxLFQ').columns


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(.corr(), annot=True, cmap='coolwarm')